In [13]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")

In [14]:
from pathlib import Path
p = Path().cwd()
print(f"original File path: {p}")
print(p)

stringpath= str(p)[0:str(p).rfind('\\')] + '\\data'
print(f"Updated File Path to pick from Data Library: { stringpath}")
ngsimfile=stringpath + '/' + 'Next_Generation_Simulation__NGSIM__Vehicle_Trajectories_and_Supporting_Data.csv'
ngsim = pd.read_csv(ngsimfile,low_memory=False)

original File path: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\scripts
c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\scripts
Updated File Path to pick from Data Library: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\data


In [15]:
filtered_ngsim= ngsim[((ngsim['Location'] == 'us-101') | (ngsim['Location'] == 'i-80')) & (ngsim['Preceding']>0) & (ngsim['Following']>0) ]
filtered_ngsim= filtered_ngsim.drop(columns=['Movement', 'Direction','Section_ID','Int_ID','D_Zone','O_Zone','Lane_ID','Following','v_Width'])


In [16]:
filtered_ngsim['Preceding_Vehicle_Class'] = np.NaN
filtered_vClass= filtered_ngsim[['Vehicle_ID','v_Class']].drop_duplicates().sort_values('v_Class')
v_Class_M=set(filtered_vClass[(filtered_vClass['v_Class'] == 1)]['Vehicle_ID'])
v_Class_C=set(filtered_vClass[(filtered_vClass['v_Class'] == 2)]['Vehicle_ID'])
v_Class_HV=set(filtered_vClass[(filtered_vClass['v_Class'] == 3)]['Vehicle_ID'])

In [17]:
filtered_ngsim["v_Class"] = filtered_ngsim["v_Class"].map({1:"Motorcycle", 2: "Car", 3: "Heavy Vehicle"})

In [18]:
for i in filtered_ngsim.index:
    if filtered_ngsim['Preceding'][i] in v_Class_C:
        filtered_ngsim['Preceding_Vehicle_Class'][i] = 'Car'
    elif filtered_ngsim['Preceding'][i] in v_Class_HV:
        filtered_ngsim['Preceding_Vehicle_Class'][i] = 'Heavy Vehicle'
    elif filtered_ngsim['Preceding'][i] in v_Class_M:
        filtered_ngsim['Preceding_Vehicle_Class'][i] = 'Motorcycle'

In [19]:
filtered_ngsim['Vehicle_combination']= filtered_ngsim['Preceding_Vehicle_Class'] + '-' + filtered_ngsim['v_Class']

In [21]:
stringpath= str(p)[0:str(p).rfind('\\')] + '\\data'
ngsimfilteredfile=stringpath + '\\' + 'Cleaned_NGSIM_Data_Part_1.csv'
filtered_ngsim.to_csv(ngsimfilteredfile, index = False)